In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.conf import SparkConf
config = SparkConf()
# config.set("property", "value")
config.setMaster("local").setAppName("StockBasic")

from pyspark.sql import SparkSession
spark =SparkSession.builder\
                    .config(conf=config)\
                    .getOrCreate()

sc= spark.sparkContext

22/03/16 21:25:06 WARN Utils: Your hostname, ubuntu-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.80.128 instead (on interface ens33)
22/03/16 21:25:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/03/16 21:25:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/16 21:25:08 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/03/16 21:25:08 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/03/16 21:25:08 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/03/16 21:25:08 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22

In [4]:
sectorDf = spark.read.format('csv')\
                .option('header', True)\
                .option('inferSchema',True)\
                .load('hdfs://localhost:9000/stocks/sectors')

sectorDf.printSchema()
sectorDf.show(2)

root
 |-- Company Name: string (nullable = true)
 |-- Industry: string (nullable = true)
 |-- Symbol: string (nullable = true)
 |-- Series: string (nullable = true)
 |-- ISIN Code: string (nullable = true)

+------------------+------------------+----------+------+------------+
|      Company Name|          Industry|    Symbol|Series|   ISIN Code|
+------------------+------------------+----------+------+------------+
|    Axis Bank Ltd.|FINANCIAL SERVICES|  AXISBANK|    EQ|INE238A01034|
|Bajaj Finance Ltd.|FINANCIAL SERVICES|BAJFINANCE|    EQ|INE296A01024|
+------------------+------------------+----------+------+------------+
only showing top 2 rows



In [9]:
sectorDf.withColumnRenamed("Company Name", "Company")\
        .withColumnRenamed("ISIN Code", "ISIN")\
        .write\
        .mode("ignore")\
        .format("jdbc")\
        .option("url", "jdbc:mysql://localhost:3306/stockdb")\
        .option("driver", "com.mysql.jdbc.Driver")\
        .option("user", "team")\
        .option("password", "Team1234!")\
        .option("dbtable", "sectors_python")\
        .save()

Wed Mar 16 21:52:17 IST 2022 WARN: Establishing SSL connection without server's identity verification is not recommended. According to MySQL 5.5.45+, 5.6.26+ and 5.7.6+ requirements SSL connection must be established by default if explicit option isn't set. For compliance with existing applications not using SSL the verifyServerCertificate property is set to 'false'. You need either to explicitly disable SSL by setting useSSL=false, or set useSSL=true and provide truststore for server certificate verification.
Wed Mar 16 21:52:17 IST 2022 WARN: Caught while disconnecting...

EXCEPTION STACK TRACE:



** BEGIN NESTED EXCEPTION ** 

javax.net.ssl.SSLException
MESSAGE: closing inbound before receiving peer's close_notify

STACKTRACE:

javax.net.ssl.SSLException: closing inbound before receiving peer's close_notify
	at sun.security.ssl.SSLSocketImpl.shutdownInput(SSLSocketImpl.java:735)
	at sun.security.ssl.SSLSocketImpl.shutdownInput(SSLSocketImpl.java:714)
	at com.mysql.jdbc.MysqlIO.quit

In [8]:
sectorDataFromMySqlDf =( spark.read
.format("jdbc")
.option("url", "jdbc:mysql://localhost:3306/stockdb")
.option("driver", "com.mysql.jdbc.Driver")
.option("user", "team")
.option("password", "Team1234!")
.option("dbtable", "sectors_python")
.load()
                       )

sectorDataFromMySqlDf.printSchema()
sectorDataFromMySqlDf.show(5)

Wed Mar 16 21:38:12 IST 2022 WARN: Establishing SSL connection without server's identity verification is not recommended. According to MySQL 5.5.45+, 5.6.26+ and 5.7.6+ requirements SSL connection must be established by default if explicit option isn't set. For compliance with existing applications not using SSL the verifyServerCertificate property is set to 'false'. You need either to explicitly disable SSL by setting useSSL=false, or set useSSL=true and provide truststore for server certificate verification.
Wed Mar 16 21:38:12 IST 2022 WARN: Caught while disconnecting...

EXCEPTION STACK TRACE:



** BEGIN NESTED EXCEPTION ** 

javax.net.ssl.SSLException
MESSAGE: closing inbound before receiving peer's close_notify

STACKTRACE:

javax.net.ssl.SSLException: closing inbound before receiving peer's close_notify
	at sun.security.ssl.SSLSocketImpl.shutdownInput(SSLSocketImpl.java:735)
	at sun.security.ssl.SSLSocketImpl.shutdownInput(SSLSocketImpl.java:714)
	at com.mysql.jdbc.MysqlIO.quit

root
 |-- Company: string (nullable = true)
 |-- Industry: string (nullable = true)
 |-- Symbol: string (nullable = true)
 |-- Series: string (nullable = true)
 |-- ISIN: string (nullable = true)

+--------------------+------------------+----------+------+------------+
|             Company|          Industry|    Symbol|Series|        ISIN|
+--------------------+------------------+----------+------+------------+
|      Axis Bank Ltd.|FINANCIAL SERVICES|  AXISBANK|    EQ|INE238A01034|
|  Bajaj Finance Ltd.|FINANCIAL SERVICES|BAJFINANCE|    EQ|INE296A01024|
|  Bajaj Finserv Ltd.|FINANCIAL SERVICES|BAJAJFINSV|    EQ|INE918I01018|
|Cholamandalam Inv...|FINANCIAL SERVICES|  CHOLAFIN|    EQ|INE121A01024|
|HDFC Asset Manage...|FINANCIAL SERVICES|   HDFCAMC|    EQ|INE127D01025|
+--------------------+------------------+----------+------+------------+
only showing top 5 rows



Wed Mar 16 21:38:12 IST 2022 WARN: Caught while disconnecting...

EXCEPTION STACK TRACE:



** BEGIN NESTED EXCEPTION ** 

javax.net.ssl.SSLException
MESSAGE: closing inbound before receiving peer's close_notify

STACKTRACE:

javax.net.ssl.SSLException: closing inbound before receiving peer's close_notify
	at sun.security.ssl.SSLSocketImpl.shutdownInput(SSLSocketImpl.java:735)
	at sun.security.ssl.SSLSocketImpl.shutdownInput(SSLSocketImpl.java:714)
	at com.mysql.jdbc.MysqlIO.quit(MysqlIO.java:2249)
	at com.mysql.jdbc.ConnectionImpl.realClose(ConnectionImpl.java:4232)
	at com.mysql.jdbc.ConnectionImpl.close(ConnectionImpl.java:1472)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRDD.close$1(JDBCRDD.scala:259)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRDD.$anonfun$compute$6(JDBCRDD.scala:268)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRDD.$anonfun$compute$6$adapted(JDBCRDD.scala:268)
	at org.apache.spark.TaskContext$$anon$1.onTaskCompletion(TaskContext